In [15]:
# public_yn이 'Y'인 데이터(공개 강의만 포함)로 EDA 수행

import pandas as pd
import json

# 1. 데이터 로드
with open('kmooc_courses_public.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)

In [16]:

# 2. 데이터 기본 정보 확인
print("--- [1] 데이터 기본 구조 ---")
df.info()  # 전체 행/열 수, 데이터 타입 확인

--- [1] 데이터 기본 구조 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6971 entries, 0 to 6970
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   6971 non-null   int64  
 1   shortname            6971 non-null   object 
 2   name                 6971 non-null   object 
 3   url                  6971 non-null   object 
 4   course_image         6971 non-null   object 
 5   org                  6971 non-null   object 
 6   org_name             6971 non-null   object 
 7   enrollment_start     6971 non-null   object 
 8   enrollment_end       6971 non-null   object 
 9   study_start          6971 non-null   object 
 10  study_end            6971 non-null   object 
 11  professor            6822 non-null   object 
 12  public_yn            6971 non-null   object 
 13  summary              6927 non-null   object 
 14  classfy_name         6971 non-null   object 
 15  middle_classfy_n

In [17]:

# 3. 결측치(Null) 확인
print("\n--- [2] 결측치 현황 ---")
print(df.isnull().sum())



--- [2] 결측치 현황 ---
id                        0
shortname                 0
name                      0
url                       0
course_image              0
org                       0
org_name                  0
enrollment_start          0
enrollment_end            0
study_start               0
study_end                 0
professor               149
public_yn                 0
summary                  44
classfy_name              0
middle_classfy_name       0
week                      0
course_playtime           0
detail_error_raw       6971
dtype: int64


In [18]:
# 4. 수치형 데이터 통계 (주차, 총 학습시간)
print("\n--- [3] 주요 수치 통계 ---")
# 문자열로 저장되어 있을 수 있으므로 숫자형으로 변환 후 요약
df['week'] = pd.to_numeric(df['week'], errors='coerce')
df['course_playtime'] = pd.to_numeric(df['course_playtime'], errors='coerce')
print(df[['week', 'course_playtime']].describe())



--- [3] 주요 수치 통계 ---
              week  course_playtime
count  6971.000000      6971.000000
mean     11.256635     69693.128676
std       4.904129     48842.572941
min       1.000000       900.000000
25%       8.000000     36000.000000
50%      14.000000     58200.000000
75%      15.000000     97200.000000
max      22.000000    457080.000000


In [19]:
# 5. 카테고리별 데이터 분포
print("\n--- [4] 대분류별 강좌 수 ---")
print(df['classfy_name'].value_counts())

print("\n--- [5] 상위 10개 제공 기관 ---")
print(df['org_name'].value_counts().head(10))



--- [4] 대분류별 강좌 수 ---
classfy_name
인문        1828
공학        1603
사회        1603
자연         488
예체능        481
의약         450
교육         293
융 · 복합     113
기타         112
Name: count, dtype: int64

--- [5] 상위 10개 제공 기관 ---
org_name
EBS         481
성균관대학교      224
고려대학교       222
이화여자대학교     204
단국대학교       185
대구대학교       179
성신여자대학교     161
경희대학교       144
호서대학교       134
부산디지털대학교    134
Name: count, dtype: int64


In [20]:
# 6. 공개 여부 확인
print("\n--- [6] 공개 여부(public_yn) 비율 ---")
print(df['public_yn'].value_counts(normalize=True) * 100)


--- [6] 공개 여부(public_yn) 비율 ---
public_yn
Y    100.0
Name: proportion, dtype: float64


In [21]:
# 7. 텍스트 임베딩 및 추천 시스템용 데이터 진단
print("--- [1] 텍스트 임베딩 핵심 필드 진단 ---")
# 텍스트가 있어야 임베딩이 가능하므로, 필수 필드 결합
df['combined_text'] = df['name'].fillna('') + " " + df['summary'].fillna('')

# 7-1. 텍스트 길이가 너무 짧은 경우 (임베딩 의미 없음)
# 이름만 있고 설명이 없는 데이터가 얼마나 되는지 확인
short_text_count = df[df['combined_text'].str.len() < 20].shape[0]
print(f"⚠️ 텍스트 20자 미만 (정보 부족): {short_text_count}건")

--- [1] 텍스트 임베딩 핵심 필드 진단 ---
⚠️ 텍스트 20자 미만 (정보 부족): 22건


In [22]:
# 7-2. 중복된 강좌명/설명 확인 (추천 결과가 중복될 위험)
duplicate_names = df[df.duplicated(['name'], keep=False)].shape[0]
print(f"⚠️ 중복된 강좌명: {duplicate_names}건 (서로 다른 ID인데 이름이 같은 경우)")

⚠️ 중복된 강좌명: 4887건 (서로 다른 ID인데 이름이 같은 경우)


In [23]:
# 8. 결측치 패턴 분석 (추천 필터링용)
print("\n--- [2] 결측치 패턴 분석 (추천 필터링용) ---")
# 추천 시스템에서 필터링 조건으로 많이 쓰는 필드들
filter_cols = ['classfy_name', 'middle_classfy_name', 'week', 'professor']
missing_patterns = df[filter_cols].isnull().sum()
print(missing_patterns)


--- [2] 결측치 패턴 분석 (추천 필터링용) ---
classfy_name             0
middle_classfy_name      0
week                     0
professor              149
dtype: int64


In [24]:
# 8-1. '분류'가 아예 없는 데이터 (카테고리 추천 불가)
no_class = df[df['classfy_name'].isnull() & df['middle_classfy_name'].isnull()].shape[0]
print(f"⚠️ 대/중분류 모두 없는 강좌: {no_class}건")

⚠️ 대/중분류 모두 없는 강좌: 0건


In [25]:
# 9. 데이터 타입 및 특수 기호 점검
print("\n--- [3] 데이터 타입 및 특수 기호 점검 ---")
# 9-1. 수치형 데이터에 0이나 음수가 있는지 (course_playtime 등)
invalid_time = df[df['course_playtime'] <= 0].shape[0]
print(f"⚠️ 학습 시간 0 이하: {invalid_time}건")


--- [3] 데이터 타입 및 특수 기호 점검 ---
⚠️ 학습 시간 0 이하: 0건


In [26]:
# 9-2. 특수문자나 HTML 잔재가 남아있는지 확인 (임베딩 노이즈)
# summary에 '<' 나 '>' 가 포함된 행 추출
html_remnants = df[df['summary'].str.contains('<|>', na=False)].shape[0]
print(f"⚠️ HTML 태그 잔재 의심: {html_remnants}건")

⚠️ HTML 태그 잔재 의심: 766건


In [27]:
# 10. summary 필드 결측된 데이터만 확인
# 상위 5개 데이터의 ID와 강좌명만 확인
missing_summary_df = df[df['summary'].isnull()]
print("\n--- 결측 강좌 샘플 (ID, 강좌명) ---")
print(missing_summary_df[['id', 'name', 'org_name']])


--- 결측 강좌 샘플 (ID, 강좌명) ---
         id                                               name       org_name
431   17494                                         빅데이터 프레임워크  고려사이버대학교(매치업)
755   17123                                유네스코 세계유산과 글로벌 시민의식          한림대학교
995   16877                                              소방지휘론     건국대학교(글로컬)
1018  16848                                        동아시아기후위기와대응          국민대학교
1075  16782                          AID+ 디지털 콘텐츠 어드밴스 : 실무 응용          한성대학교
1076  16781                            AID+ 디지털 콘텐츠 프로 : 실무 개발          한성대학교
1077  16780                           AID+ 디지털 콘텐츠 베이직 : 실무 설계          한성대학교
1272  16496                                         빅데이터 프레임워크  고려사이버대학교(매치업)
1671  15776                                              소방지휘론     건국대학교(글로컬)
1699  15738  ទេសចរណ៍វប្បធម៌កូរេជាមួយនឹងជនបរទេស [외국인과 함께하는 한...          한성대학교
1833  15537                                유네스코 세계유산과 글로벌 시민의식          한림대학교
2055  15195                         